In [ ]:
import numpy as np
from numba import jit

import matplotlib.pyplot as plt
from functools import partial

import sys
sys.path.append('../miniMD')
from miniMD import *

# Rare Events and Transition State Theory

## Rare but Important Events

The goal of this exercise is a first approach to rate calculation for rare events. Rare events are processes that happen rarely, but when they happen their dynamics is fast, in comparison to the molecular motion. This is a challenge for the numerical simulations, as a high resolution is needed for resolving the event but most of the computational time is spent in simulating non-interesting portion of the configuration/phase space.

In this notebook we discuss two approaches: 

We first aim to calculate the correlation function $c_{AB}(t)$ from which we can estimate the rate based on data from a long MD simulation. For higher barriers and more complex systems this is usually not feasible and one would perform transition path sampling to obtain $c_{AB}(t)$ (we'll have a look at that in the next exercise).

We then proceed to estimate the rate using an approximation based on transition state theory. Here we estimate the rate based on the volume of a state, the probability to be at the barrier top and the velocities at the barrier top.

As in previous cases we focus on a test system, the symmetric double well potential in 2D that we now are familiar with. We define functions to compute potential energy and forces:

In [ ]:

@jit(nopython=True) 
def custom_potential_energy(current_x : np.ndarray) -> float:
    """
    Calculates the potential energy given a configuration current_x. 
    
    Parameters
    ----------
    current_x : np.ndarray
        Current configuration to be propagated. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    Returns
    -------
    U : float
        Potential energy of the configuration

    """
    return 10 * ((current_x[0]**2 - 1)**2 + (current_x[0] - current_x[1])**2)


@jit(nopython=True) 
def custom_force_function(current_x : np.ndarray) -> np.ndarray:
    """
    Calculates the force given a configuration current_x. 

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration to be propagated. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    Returns
    -------
    force : np.ndarray
        Force corresponding the provided configuration.

    """
    force = np.zeros(2)
    
    force[0] = -2 * 10 * (2 * current_x[0]**3 -current_x[0] - current_x[1])
    force[1] = 2 * 10 * (current_x[0] - current_x[1])

    return force


A plot of the potential energy surface for this particular model is shown below.

In [ ]:
x_values = np.linspace(-2.6, 2.6, 200)
y_values = np.linspace(-2.6, 2.6, 200)

x_grid, y_grid = np.meshgrid(x_values, y_values)
energies = np.zeros((len(x_values), len(y_values)))

for i in range(len(x_values)):
    for j in range(len(y_values)):

        energies[i,j] = custom_potential_energy(np.array([x_grid[i, j], y_grid[i, j]]))


In [ ]:
fig, ax = plt.subplots(1, figsize=(4,4), dpi=180)

ax.contourf(x_grid, y_grid, energies, levels=np.linspace(0, 20, 10), cmap="RdBu_r")

ax.set_xlabel(r"$x_0$")
ax.set_ylabel(r"$x_1$")

plt.show()

We can now perform a molecular dynamics simulation for this system through the usual code snippets for overdamped Langevin Molecular Dynamics. We raise the temperature a bit to see more transitions, but feel free to play with this parameter:

In [ ]:
total_steps = 2000000
equilibration_steps = 1000
output_frequency = 1

beta = 0.5
timestep = 0.001 # 1 fs = 0.001 ps
diffusion_coefficient = 1

initial_x =- np.ones(2)

assert equilibration_steps < total_steps, "Make sure you don't equilibrate longer than you simulate."
assert output_frequency < total_steps, "Make sure you don't output less often than you simulate."
assert output_frequency > 0, "The output frequency needs to be larger than 0"


In [ ]:
previous_x = initial_x.copy()

total_frames = int(np.ceil((total_steps - equilibration_steps) / output_frequency))
trajectory = np.zeros((total_frames , 2))

for step in range(total_steps):

    current_force = custom_force_function(previous_x)
    
    previous_x = update_positions(previous_x, current_force, beta, timestep, diffusion_coefficient)

    if step >= equilibration_steps and step % output_frequency == 0:
        index = (step - equilibration_steps) // output_frequency
        trajectory[index] = previous_x

As already the case in other exercises, we would like to study the behaviour of the system along a collective variable $\xi(x)$, where by $x = (x_0, x_1)$ we indicate a configuration of the system in two dimensions. For this exercise we choose the collective variable $\xi(x) = x_0 + x_1$.

In [ ]:
@jit(nopython=True) 
def custom_cv(current_x : np.ndarray) -> float:
    """
    Calculates the collective variable zeta given a configuration current_x.

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    Returns
    -------
    zeta : float
        CV corresponding the provided configuration.
    """

    return current_x[..., 0] + current_x[..., 1]

Once a function for the collective variable is implemented let us project the trajectory computed via standard MD on the designated CV. From this, we extract the free energy by computing the probability along the CV and then by Boltzmann inversion.

In [ ]:
trajectory_cv = custom_cv(trajectory)

hist, bins = np.histogram(trajectory_cv, bins=100, density=True)
bin_centers = (bins[1:] + bins[:-1]) / 2

free_energy = -1/beta*np.log(hist)
free_energy -= np.min(free_energy)

In order to compute rates and to confront them with real data from experiments, it is important to obtain values that are in physical units. For this reason, let us define the simulation time in some submultiple of seconds. Recall that the timestep used by default in these exercises is given in picoseconds: $\delta t = 0.001 \text{ps} = 1 \text{fs}$

In [ ]:
simulation_time = np.arange(total_frames) * timestep * output_frequency

In the following plotting script we show the trajectory, the probability density and the free energy along the collective variable $\xi$.

In [ ]:
# Plotting
fig, ax = plt.subplots(1, 3, figsize=(10,4), dpi=180, gridspec_kw={'width_ratios': [2, 1, 1]}, sharey=True)

ax[0].plot(simulation_time, trajectory_cv, lw=1)
ax[0].hlines(y=-2, xmin=0, xmax=simulation_time[-1], ls="--", color="k", alpha=0.25)
ax[0].hlines(y=2, xmin=0, xmax=simulation_time[-1], ls="--", color="k", alpha=0.25)
ax[0].hlines(y=0, xmin=0, xmax=simulation_time[-1], ls=":", color="k", alpha=0.25)
ax[0].set_ylabel(r"$\xi$")
ax[0].set_xlabel("Time [ps]")
ylim = ax[0].get_ylim()

ax[1].plot(hist, bin_centers)
ax[1].hlines(y=-2, xmin=0, xmax=np.max(hist), ls="--", color="k", alpha=0.25)
ax[1].hlines(y=2, xmin=0, xmax=np.max(hist), ls="--", color="k", alpha=0.25)
ax[1].hlines(y=0, xmin=0, xmax=np.max(hist), ls=":", color="k", alpha=0.25)
ax[1].set_xlabel(r"$P(\xi)$")

ax[2].plot(free_energy, bin_centers)
ax[2].hlines(y=-2, xmin=0, xmax=np.max(free_energy), ls="--", color="k", alpha=0.25)
ax[2].hlines(y=2, xmin=0, xmax=np.max(free_energy), ls="--", color="k", alpha=0.25)
ax[2].hlines(y=0, xmin=0, xmax=np.max(free_energy), ls=":", color="k", alpha=0.25)
ax[2].set_xlabel(r"$F(\xi) = -kT \log P(\xi)$")

plt.twinx()
plt.ylim(ylim)
plt.yticks([-2, 0, 2], [r"$\xi_A$", r"$\xi^*$", r"$\xi_B$"])

plt.show()

From this analysis we see how the free energy landscape presents two minima corresponding to $\xi_A = -2$ and $\xi_B = 2$ (highlighted by the dashed lines in the plots) and a barrier between them corresponding to $\xi^*=0$ (highlighted by the dotted line in the plots). We can use these coordinates to define some stable states via a boolean function that takes as input the current configuration `current_x` a `callable` object `cv_function` to represent the particular CV used and the bounds (a tuple of `floats`) in the CV space that limit the states.

**1) Complete the custom_bounded_state function based on the template given below.**

In [ ]:
@jit(nopython=True) 
def custom_bounded_state(current_x : np.ndarray, cv_function : callable, bounds : tuple = (-np.inf, np.inf)  ) -> bool:
    """
    Returns a bool or bool array which is true if current_x is within in the bounds.

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    cv_function : callable
        Function that accepts the current configuration and maps it onto the collective variable.
        
    bounds : (float, float)
        The lower and upper bounds for x to be considered within the stable state


    Returns
    -------
    inState : bool
        True for each x in current_x if x is within the bounds
    """

    cv_values = cv_function(current_x)
    
    inState = (cv_values > bounds[0]) & (cv_values < bounds[1])
    
    return inState

We can then use once again the `partial` constructor from `functools` to obtain a `state_A_indicator` and a `state_B_indicator` that only take only the `current_x` and return `True` if `current_x` is in state A (B) and `False` otherwise. These are the functions that are commonly indicate by $h_A(x)$ and $h_B(x)$.

In [ ]:
state_A_indicator = partial(custom_bounded_state, cv_function=custom_cv, bounds=(-np.inf, 0))
state_B_indicator = partial(custom_bounded_state, cv_function=custom_cv, bounds=(0, np.inf))

With the following plotting script we can visualize the stable states over the potential energy surface represented by our model.

In [ ]:
x_values = np.linspace(-2.6, 2.6, 200)
y_values = np.linspace(-2.6, 2.6, 200)

x_grid, y_grid = np.meshgrid(x_values, y_values)
in_state_A = np.zeros((len(x_values), len(y_values)))
in_state_B = np.zeros((len(x_values), len(y_values)))

for i in range(len(x_values)):
    for j in range(len(y_values)):

        in_state_A[i,j] = state_A_indicator(np.array([x_grid[i, j], y_grid[i, j]]))
        in_state_B[i,j] = state_B_indicator(np.array([x_grid[i, j], y_grid[i, j]]))

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,4), dpi=180)

ax.contourf(x_grid, y_grid, energies, levels=np.linspace(0, 20, 10), cmap="RdBu_r")
ax.contourf(x_grid, y_grid, in_state_A, cmap="Blues", alpha=0.1)
ax.contourf(x_grid, y_grid, in_state_B, cmap="Oranges", alpha=0.1)

ax.text(0.625, 0.1, "State A", transform=ax.transAxes, c="C0")
ax.text(0.125, 0.9, "State B", transform=ax.transAxes, c="C1")

ax.set_xlabel("x")
ax.set_ylabel("y")

plt.show()

Using the state indicator functions it is also possible to visualize how the system jumps from one state to the other.

In [ ]:
state_A_trajectory = state_A_indicator(trajectory)
state_B_trajectory = state_B_indicator(trajectory)

If we plot the value of $h_A(x)$ along the trajectory we can clearly see when the system is in $A$ and when it is in $B$. Note that there is no no-man's land in our definition of state and this means that $h_A(x) + h_B(x) = 1 \ \forall x$. In other words the system is either in $A$ or in $B$. This can be seen in the second plot of the following cell where the values of $h_A(x)$ and $h_B(x)$ are shown for the first $100 \text{ps}$ of simulation. In the plot on the left the value of $h_A(x)$ is shown for the entire simulation.

In [ ]:
# Plotting
fig, ax = plt.subplots(1, 2, figsize=(10,4), dpi=180, sharey=True)

ax[0].plot(simulation_time, state_A_trajectory, lw=1)
ax[0].set_ylabel(r"$h_A(x)$")
ax[0].set_xlabel("Time [ps]")

ax[1].plot(simulation_time, state_A_trajectory, lw=1, ls=":")
ax[1].plot(simulation_time, state_B_trajectory, lw=1, ls=":")
ax[1].set_xlim(0,100)
ax[1].set_xlabel("Time [ps]")

plt.show()

We can now use the theory developed during the course to find the rate of the process. We know from the microscopic theory that there should be a regime $\tau_{mol} < t < \tau_{rxn}$ ($\tau_{mol}$ being the molecular timescale, $\tau_{rxn}$ being the reaction time) where the correlation function
$$
C_{AB}(t) = \frac{\langle h_A(0)h_B(t)\rangle}{\langle h_A \rangle}
$$
increases linearly with time with slope $k_{AB}$. In other words
$$
k(t) = \frac{d}{dt}C_{AB}(t)\approx k_{AB} \quad \text{for} \quad \tau_{mol} < t < \tau_{rxn}
$$
In the next cell we want to compute the correlation function and find the linear regime to compute $k_{AB}$.

**2) Estimate $C_{AB}(t)$ from the simulation data.** 

TIP: Initialize an array h_AB of size max_frame_lag. For every point on the trajectory, first check if we are in state A. If we are, for each point up until max_frame_lag from the current point see if we are in state B. If we are, add 1 to the respecting position of h_AB. Later you need to divide h_AB by the number of total points to obtain the average h_AB. 

In [ ]:
max_frame_lag = 300

time = np.arange(max_frame_lag) * timestep * output_frequency
h_AB = np.zeros(max_frame_lag)


for frame_index in range(trajectory.shape[0] - h_AB.shape[0]):

    ...

average_h_AB = ...
avg_h_A = ...

c_AB = average_h_AB / avg_h_A

Below we plot the correlation function and its derivative.

**3) Tune the horizontal line to find your plateau value indicating the rate.**

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,4), dpi=180)

plt.plot(time, c_AB)
plt.ylabel(r"$c_{AB}(t)$")
plt.xlabel("Time [ps]")

plt.twinx()
plt.plot(time[1:-1], (c_AB[2:] - c_AB[:-2]) / (time[2:] - time[:-2]), c="C1")

# Tune here
plt.axhline(... , color="k", ls="dashed", alpha=0.25)

plt.ylabel(r"d/dt $c_{AB}$ [1/ps]")

plt.show()

## Transition State Theory

A different route to get an estimate (an upper bound, in fact) of the transition rate $k_AB$ is to use transition state theory, which estimates the rate of transition as
$$
k_{TST} = \frac{1}{2}\left\langle|\dot{\xi}| \right\rangle_{\xi=\xi^*}\frac{\exp[-\beta F_{\xi}(\xi^*)]}{\int_{-\infty}^{\xi^*}\text{d}\xi'\exp[-\beta F_{\xi}(\xi')]}
$$
This approach is very interesting as it only involves static calculation and no dynamical trajectories are needed. On the other hand, the term
$$
\left\langle|\dot{\xi}| \right\rangle_{\xi=\xi^*}
$$
implicitly assumes that every trajectory that flies out from the dividing surface that identifies the free energy barrier reaches a stable state without recrossing. This produces an overestimation of the transition rate, i.e. we have that $k_{TST} \geq k_{AB}$.

The free energy necessary to compute the numerator and the denominator can be obtained from standard techniques, i.e. simple Boltzmann inversion when the system allows it, or more advanced techniques, like umbrella sampling or metadynamics.

Lets first remind ourselves of the free energy we obtained along $\xi$ in the above simulation:

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,3), dpi=180)

ax.plot(bin_centers, free_energy)
ax.vlines(x=0, ymin=0, ymax=np.max(free_energy), ls="--", color="k", alpha=0.25)

ax.set_xlabel(r"$\xi'$")
ax.set_ylabel(r"$F_{\xi}(\xi')$")

plt.show()

**4) For obtaining the numerator and denominator, extract the Boltzmann factor at the barrier top and integrate the free energy to obtain the "volume" of state A.** TIP: Be careful with NaNs in the free energy, you can discard these points that usually occur on the edges.

In [ ]:
xi_star_indx = ...
num = ...

In [ ]:
free_energy_cut = free_energy[: ... ] # Lets cut away everything belonging to state B
denom = ...

The last term missing, i.e.
$$
\left\langle|\dot{\xi}| \right\rangle_{\xi=\xi^*}
$$
can be obtained running a bunch of very short trajectories all starting from the barrier top, i.e. $\xi^* = 0$. The time derivative can be caluclated numerically and the average can be carried out. Note that the initial conditions for the fleeting trajectories need to be distributed physically, i.e. according to the Boltzmann distribution. To do this, a small Monte Carlo simulation can be implemented to obtained Boltzmann distributed configurations along the line $\xi^* = 0$.


In [ ]:
total_steps = 2 # two steps are enough to compute velocities
equilibration_steps = 0 # no equilibration is needed if initial conditions are Boltzmann distributed
output_frequency = 1
n_fleeting_trajs = 1000

beta = 0.3
timestep = 0.001
diffusion_coefficient = 1

initial_x = np.zeros(2)

assert equilibration_steps < total_steps, "Make sure you don't equilibrate longer than you simulate."
assert output_frequency < total_steps, "Make sure you don't output less often than you simulate."
assert output_frequency > 0, "The output frequency needs to be larger than 0"


**5) Complete the code below to start many short trajectories from the dividing surface.** While we provide a code to obtain a new point, your part is to implement the MD code and the calculation of the absolute velocity on xi.

In [ ]:
total_frames = int(np.ceil((total_steps - equilibration_steps) / output_frequency))

fleeting_trajectory = np.zeros((n_fleeting_trajs, total_frames+1, 2)) # Should contain the trajectories
abs_dot_xi = np.zeros((n_fleeting_trajs, total_frames-1)) # Should contain the velocities on xi

frame_step = timestep*output_frequency

acc = 0

for traj in range(n_fleeting_trajs):

    previous_x = initial_x.copy()

    fleeting_trajectory[traj][0] = previous_x
    
    for step in range(total_steps):

        current_force = custom_force_function(previous_x)
        
        previous_x = update_positions(previous_x, current_force, beta, timestep, diffusion_coefficient)

        if step >= equilibration_steps and step % output_frequency == 0:
            index = (step - equilibration_steps) // output_frequency
            fleeting_trajectory[traj][index+1] = previous_x
    
    
    fleeting_trajectory_cv = ...
    abs_dot_xi[traj] = ...


    # We do MC here to obtain a new initial point for the next trajectory
    x_new = initial_x[0] + 0.75*np.random.uniform(low=-1, high=1)
    y_new = -x_new
    proposed_x = np.array([x_new, y_new])
    e_old = custom_potential_energy(previous_x)
    e_new = custom_potential_energy(proposed_x)

    test = np.random.uniform(low=0, high=1)
    if test < np.exp(-beta*(e_new-e_old)):
        initial_x = proposed_x.copy()
        acc += 1

print(f"MC acceptance ratio: {acc/n_fleeting_trajs}")

In [ ]:
abs_dot_xi_avg = abs_dot_xi.mean()

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,4), dpi=180)

ax.contourf(x_grid, y_grid, energies, levels=np.linspace(0, 20, 10), cmap="RdBu_r")
ax.contourf(x_grid, y_grid, in_state_A, cmap="Blues", alpha=0.1)
ax.contourf(x_grid, y_grid, in_state_B, cmap="Oranges", alpha=0.1)

for traj in range(n_fleeting_trajs):
    ax.scatter(fleeting_trajectory[traj][0,0], fleeting_trajectory[traj][0,1], c="r", s=2, alpha=0.1, zorder=10)
    ax.plot(fleeting_trajectory[traj][:,0], fleeting_trajectory[traj][:,1], lw=0.2)

ax.text(0.625, 0.1, "State A", transform=ax.transAxes, c="C0")
ax.text(0.125, 0.9, "State B", transform=ax.transAxes, c="C1")

ax.set_xlabel("x_0")
ax.set_ylabel("x_1")

plt.show()

**5) Calculate the rate using the TST approximation and compare it to the value obtained from $C_{AB}(t)$.** 

In [ ]:
k_AB = ...

In [ ]:
print(f"Transition Rate for A to B: {k_AB} 1/ps")